In [ ]:
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re
import codecs
import json

In [ ]:
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag] = tags[elem.tag] + 1
        else:
            tags[elem.tag] = 1
        elem.clear()
    return tags

In [ ]:
OSMFILE = 'sample.osm'

In [ ]:
print(count_tags(OSMFILE))

In [ ]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [ ]:
def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if lower.match(k):
            keys['lower'] = keys['lower'] + 1
        elif lower_colon.search(k):
            keys['lower_colon'] = keys['lower_colon'] + 1
        elif problemchars.search(k):
            keys['problemchars'] = keys['problemchars'] + 1
        else:
            keys['other'] = keys['other'] + 1
    
    return keys

In [ ]:
def process_map(filename):
    keys = {'lower': 0, 'lower_colon': 0, 'problemchars': 0, 'other': 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    
    return keys

In [ ]:
print(process_map(OSMFILE))

In [ ]:
def get_user(element):
    return element.attrib['uid']

In [ ]:
def fetch_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            users.add(get_user(element))
            
    return users

In [ ]:
print(fetch_users(OSMFILE))

In [ ]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", 
            "Place", "Square", "Lane", "Road", "Trail",
            "Parkway", "Commons"]

mapping = {}